In [14]:
from llmexp.llm.smollm import LLMWrapper
from accelerate import Accelerator
import torch

# checkpoint = "meta-llama/Meta-Llama-3-8B-Instruct"
checkpoint = "meta-llama/Llama-3.2-1B-Instruct"
# checkpoint = "HuggingFaceTB/SmolLM-1.7B-Instruct"
# saved_mab_model = "checkpoints/mab_model_100.pth"
saved_mab_model = "checkpoints/mab_model_120.pth"


accelerator = Accelerator()
device = accelerator.device


llm = LLMWrapper(checkpoint, device=device)
tokenizer = llm.tokenizer

In [15]:
# instruction = "Analyze the sentiment of the following sentence. Be brief."
# instruction = "Analyze the sentiment of the following sentence and respond with only one word: 'positive', 'negative', or 'neutral', based on the overall tone and meaning of the sentence, if no enough information provided, respond with 'not clear' with an explanation."
instruction = "Analyze the sentiment of the following sentence and respond concisely."
# user_input = "I am extremely disappointed with the quality; it broke after just one day."
user_input = "The service at this restaurant was fantastic, and the staff were so friendly."
# user_input = "<|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|><|eot_id|>"
# user_input = "I'm so happy with the product!"
# user_input = "The bright sunshine and gentle breeze made my afternoon truly delightful."
# user_input = "I felt deeply disappointed and frustrated after the meeting went completely off track."
# user_input = "Although the food at the restaurant was excellent, the service left much to be desired."


content = [
            {"role": "system", 
            "content": instruction
            },

            {"role": "sentence", 
            "content": user_input
            }
        ]
template = tokenizer.apply_chat_template(content, tokenize=False, add_generation_prompt=True)
# print(template)

# The generated outputs 
gen_output = llm.generate_from_texts(template)
print(template)
print(gen_output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Analyze the sentiment of the following sentence and respond concisely.<|eot_id|><|start_header_id|>sentence<|end_header_id|>

The service at this restaurant was fantastic, and the staff were so friendly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Analyze the sentiment of the following sentence and respond concisely.<|eot_id|><|start_header_id|>sentence<|end_header_id|>

The service at this restaurant was fantastic, and the staff were so friendly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The sentiment of this sentence is overwhelmingly positive. The use of words like "fantastic" and "friendly" convey a high level of satisfaction and appreciation for the service.<|eot_id|>


In [16]:
from llmexp.utils.data_utils import DataCollatorSST2
data_collator = DataCollatorSST2(tokenizer, max_length=512, instruction=instruction)

example = {
    'sentence': user_input,
    'label': 1
}

example = data_collator([example]).to(device)
print(example)


{'input_ids': tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,  10263,    220,   2366,     19,    271,   2127,  56956,
            279,  27065,    315,    279,   2768,  11914,    323,   6013,   3613,
            285,    989,     13, 128009, 128006,  52989, 128007,    271,    791,
           2532,    520,    420,  10960,    574,  14964,     11,    323,    279,
           5687,   1051,    779,  11919,     13, 128009, 128006,  78191, 128007]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0'), 'context_mask': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [17]:
print(tokenizer.decode(example['input_ids'][0] * example['attention_mask'][0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Analyze the sentiment of the following sentence and respond concisely.<|eot_id|><|start_header_id|>sentence<|end_header_id|>

The service at this restaurant was fantastic, and the staff were so friendly.<|eot_id|><|start_header_id|>assistant<|end_header_id|>


In [18]:
gen_output = llm.generate(example['input_ids'], example['attention_mask'])
print(gen_output)
print(tokenizer.decode(gen_output['input_ids'][0]))

{'input_ids': tensor([[128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,   2696,
             25,   6790,    220,   2366,     18,    198,  15724,   2696,     25,
            220,   1627,  10263,    220,   2366,     19,    271,   2127,  56956,
            279,  27065,    315,    279,   2768,  11914,    323,   6013,   3613,
            285,    989,     13, 128009, 128006,  52989, 128007,    271,    791,
           2532,    520,    420,  10960,    574,  14964,     11,    323,    279,
           5687,   1051,    779,  11919,     13, 128009, 128006,  78191, 128007,
            271,    791,  27065,    315,    420,  11914,    374,  55734,   6928,
             13,    578,   1005,    315,   4339,   1093,    330,  61827,   5174,
              1,    323,    330,  82630,      1,  20599,    264,   1579,   2237,
            315,  24617,    323,  35996,    369,    279,   2532,     13, 128009]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [19]:
from llmexp.explainer.mab_model import MABModel
mab_model = MABModel.load_with_base_model(torch.load(saved_mab_model), llm, hidden_size=1024)
mab_model.to(device)
print()

/tmp/1211681.1.gpu/ipykernel_1895354/3922162976.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mab_model = MABModel.load_with_base_model(torch.load(saved_mab_model), ll

In [20]:
# Get gen_start_index 
gen_start_index = example['input_ids'].shape[1]
print(gen_start_index)

63


In [21]:
input_ids = gen_output['input_ids']
attention_mask = gen_output['attention_mask']
# logits, values = mab_model.get_logits_value(input_ids, attention_mask)
logits_list = mab_model.inference(input_ids, attention_mask, gen_start_index)


# mab_values = torch.softmax(logits, dim=-1)

context_mask = example['context_mask']
mab_values_list = []
context_mask_list = []
for logits in logits_list:
    # Get the size difference
    pad_size = logits.size(1) - context_mask.size(1)
    # Right pad context_mask with False values
    padded_context_mask = torch.nn.functional.pad(context_mask, (0, pad_size), value=1)

    masked_logits = logits.masked_fill(~padded_context_mask.bool(), float('-inf'))
    mab_values = torch.softmax(masked_logits, dim=-1)

    # masked_logits = logits.masked_fill(~padded_context_mask.bool(),0)
    # mab_values = masked_logits
    # mab_values = torch.softmax(masked_logits, dim=-1)

    mab_values_list.append(mab_values)
    context_mask_list.append(padded_context_mask)

In [22]:
mab_values_list

[tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0296,
          0.0363, 0.0277, 0.0412, 0.0218, 0.0702, 0.0100, 0.0583, 0.0321, 0.0499,
          0.1211, 0.0826, 0.3211, 0.0193, 0.0787, 0.0000, 0.0000, 0.0000, 0.0000]],
        device='cuda:0'),
 tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.00

In [23]:
def visualize_tokens_with_values(input_ids, mab_values, context_mask, tokenizer):
    # Decode tokens one by one to preserve alignment
    tokens = []
    for i in range(input_ids.shape[1]):
        token = tokenizer.decode(input_ids[0, i:i+1])
        tokens.append(token)
    
    # Normalize MAB values to [0,1] for color intensity first
    mab_values = mab_values * context_mask
    # Create a mask for non-zero values
    non_zero_mask = mab_values[0] != 0
    # normalized_values = (mab_values[0] - mab_values[0].min()) / (mab_values[0].max() - mab_values[0].min())
    normalized_values = torch.zeros_like(mab_values[0])
    # Only normalize non-zero values
    if non_zero_mask.any():  # Check if there are any non-zero values
        non_zero_values = mab_values[0][non_zero_mask]
        normalized_non_zero = (non_zero_values - non_zero_values.min()) / (non_zero_values.max() - non_zero_values.min())
        normalized_values[non_zero_mask] = normalized_non_zero
    
    # Pad normalized_values with a zero at the end
    padded_normalized_values = torch.cat([normalized_values, torch.zeros(1, device=mab_values.device)], dim=0)
    # Pad original mab_values with the last actual value
    padded_mab_values = torch.cat([mab_values[0], mab_values[0][-1:]], dim=0)
    
    # Generate HTML with colored text and values
    html_output = "<div style='font-family: monospace; line-height: 2; background-color: white; padding: 10px;'>"
    for token, value, orig_value in zip(tokens, padded_normalized_values, padded_mab_values):
        # Use a gradient from white to green
        intensity = float(value)
        green_color = int(intensity * 200)  # Control the maximum intensity
        html_output += f'<span style="color: black; background-color: rgba(0, {green_color}, 0, 0.3); padding: 0.2em; margin: 0.1em; border-radius: 3px;" title="MAB: {orig_value:.3f}, Norm: {value:.3f}">{token}</span>'
    html_output += "</div>"
    
    # Print the values
    # print("Token\tNormalized Value\tOriginal MAB Value")
    # print("-" * 50)
    # for token, value, orig_value in zip(tokens, padded_normalized_values, padded_mab_values):
    #     print(f"{token}\t{value:.3f}\t\t{orig_value:.3f}")
    
    from IPython.display import HTML
    return HTML(html_output)

In [24]:
idx = 2
logits_list[idx]

tensor([[0.5162, 0.5035, 0.5275, 0.4990, 1.2589, 0.8307, 0.6754, 1.9452, 1.5127,
         0.8182, 0.8975, 0.4900, 1.5201, 0.5174, 0.7146, 0.3863, 0.9935, 0.9886,
         0.7439, 1.4511, 0.6320, 1.0093, 1.5257, 0.9213, 0.8336, 0.5626, 0.6095,
         0.2221, 3.0601, 1.0711, 0.6989, 1.0166, 0.9326, 1.6393, 2.0353, 1.1427,
         1.3284, 1.4820, 1.8112, 0.6748, 0.5805, 0.2200, 0.7383, 0.7209, 0.5135,
         0.4791, 0.4902, 0.4231, 0.4662, 1.7824, 0.2752, 1.3872, 0.6124, 0.7128,
         0.4333, 1.0479, 1.7916, 0.0632, 0.5045, 0.7898, 0.7956, 1.1635, 0.8410,
         1.9600, 1.6973]], device='cuda:0')

In [25]:
# Usage:
for idx in range(len(logits_list)):
    mab_values = mab_values_list[idx]
    visualization = visualize_tokens_with_values(input_ids, mab_values, context_mask_list[idx], tokenizer)
    display(visualization)

In [26]:
input_ids.shape

torch.Size([1, 99])